In [1]:
import kagglehub

dataset = kagglehub.dataset_download("ahindoruca/data-v2-aug")

In [1]:
!mkdir /kaggle/working/data
!cp -r /kaggle/input/data-v2-aug/data_v2_aug/* /kaggle/working/data

In [2]:
!cat /kaggle/working/data/data.yaml

names:
- dentigeroz kist
- keratokist
- radikuler kist
- ameloblastoma
- odontoma
nc: 5
path: /Users/sahindogruca/Desktop/code/ytü/semester-7/computerProject2/data/data_v2_aug
test: test/images
train: train/images
val: valid/images


In [3]:
%%writefile /kaggle/working/data.yaml
path: /kaggle/working/data
train: train/images
val: valid/images
test: test/images

names:
- dentigeroz kist
- keratokist
- radikuler kist
- ameloblastoma
- odontoma
nc: 5

Writing /kaggle/working/data.yaml


In [4]:
import json
import os
import yaml
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import cv2
import numpy as np
import shutil


def load_yaml(yaml_path):
    """Load YAML configuration file"""
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    return data


def polygon_to_bbox(polygon):
    """
    Convert polygon coordinates to bounding box [x, y, width, height]
    polygon: list of [x1, y1, x2, y2, ..., xn, yn] (normalized 0-1)
    """
    x_coords = polygon[0::2]
    y_coords = polygon[1::2]
    
    x_min = min(x_coords)
    y_min = min(y_coords)
    x_max = max(x_coords)
    y_max = max(y_coords)
    
    width = x_max - x_min
    height = y_max - y_min
    
    return [x_min, y_min, width, height]


def calculate_area(polygon, img_width, img_height):
    """Calculate polygon area in pixel coordinates"""
    # Convert normalized coordinates to pixel coordinates
    points = []
    for i in range(0, len(polygon), 2):
        x = polygon[i] * img_width
        y = polygon[i + 1] * img_height
        points.append([x, y])
    
    points = np.array(points, dtype=np.int32)
    area = cv2.contourArea(points)
    return float(area)


def convert_yolo_to_coco(data_yaml_path, output_root):
    """
    Convert YOLO polygon dataset to COCO format with standard directory structure
    
    Args:
        data_yaml_path: Path to data.yaml file
        output_root: Root directory for output (will create dataset/ structure)
    """
    # Load configuration
    config = load_yaml(data_yaml_path)
    dataset_path = Path(config['path'])
    class_names = config['names']
    num_classes = config['nc']
    
    print(f"Dataset path: {dataset_path}")
    print(f"Number of classes: {num_classes}")
    print(f"Classes: {class_names}")
    
    # Create standard COCO directory structure
    output_root = Path(output_root)
    images_dir = output_root / "images"
    annotations_dir = output_root / "annotations"
    
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(annotations_dir, exist_ok=True)
    
    print(f"\nCreating COCO dataset at: {output_root}")
    print(f"  images/")
    print(f"  annotations/")
    
    # Process each split (train, val, test)
    splits = []
    if 'train' in config:
        splits.append(('train', config['train']))
    if 'val' in config:
        splits.append(('val', config['val']))
    if 'test' in config:
        splits.append(('test', config['test']))
    
    for split_name, split_path in splits:
        print(f"\n{'='*60}")
        print(f"Processing {split_name} split...")
        print('='*60)
        
        # Create split-specific image directory
        split_images_dir = images_dir / split_name
        os.makedirs(split_images_dir, exist_ok=True)
        
        # Initialize COCO format structure
        coco_data = {
            "images": [],
            "annotations": [],
            "categories": []
        }
        
        # Add categories
        for i, class_name in enumerate(class_names):
            coco_data["categories"].append({
                "id": i,
                "name": class_name,
                "supercategory": "object"
            })
        
        # Get image directory from original dataset
        images_src = dataset_path / split_path
        
        # Determine labels directory
        if 'train' in split_path:
            labels_dir = dataset_path / 'train' / 'labels'
        elif 'valid' in split_path or 'val' in split_path:
            labels_dir = dataset_path / 'valid' / 'labels'
        elif 'test' in split_path:
            labels_dir = dataset_path / 'test' / 'labels'
        else:
            labels_dir = images_src.parent / 'labels'
        
        print(f"Source images: {images_src}")
        print(f"Source labels: {labels_dir}")
        print(f"Destination images: {split_images_dir}")
        
        if not images_src.exists():
            print(f"Warning: {images_src} does not exist, skipping...")
            continue
        
        # Get all image files
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
        image_files = []
        for ext in image_extensions:
            image_files.extend(list(images_src.glob(f'*{ext}')))
            image_files.extend(list(images_src.glob(f'*{ext.upper()}')))
        
        print(f"Found {len(image_files)} images")
        
        annotation_id = 1
        
        # Process each image
        for image_id, image_path in enumerate(tqdm(image_files, desc=f"Converting {split_name}")):
            # Copy image to new location
            dest_image_path = split_images_dir / image_path.name
            if not dest_image_path.exists():
                shutil.copy2(image_path, dest_image_path)
            
            # Load image to get dimensions
            try:
                img = Image.open(image_path)
                img_width, img_height = img.size
            except Exception as e:
                print(f"Error loading image {image_path}: {e}")
                continue
            
            # Add image info (use just filename, path will be images/split/)
            coco_data["images"].append({
                "id": image_id,
                "file_name": image_path.name,
                "width": img_width,
                "height": img_height
            })
            
            # Get corresponding label file
            label_path = labels_dir / f"{image_path.stem}.txt"
            
            if not label_path.exists():
                # No annotations for this image
                continue
            
            # Read label file
            try:
                with open(label_path, 'r') as f:
                    lines = f.readlines()
            except Exception as e:
                print(f"Error reading label {label_path}: {e}")
                continue
            
            # Process each annotation
            for line in lines:
                parts = line.strip().split()
                if len(parts) < 7:  # At least class_id + 3 points (6 coordinates)
                    continue
                
                class_id = int(parts[0])
                
                # Extract polygon coordinates (normalized)
                polygon = [float(x) for x in parts[1:]]
                
                # Convert normalized coordinates to pixel coordinates for COCO
                segmentation = []
                for i in range(0, len(polygon), 2):
                    x = polygon[i] * img_width
                    y = polygon[i + 1] * img_height
                    segmentation.append(x)
                    segmentation.append(y)
                
                # Calculate bounding box from polygon (normalized)
                bbox_norm = polygon_to_bbox(polygon)
                
                # Convert bbox to pixel coordinates [x, y, width, height]
                bbox = [
                    bbox_norm[0] * img_width,
                    bbox_norm[1] * img_height,
                    bbox_norm[2] * img_width,
                    bbox_norm[3] * img_height
                ]
                
                # Calculate area
                area = calculate_area(polygon, img_width, img_height)
                
                # Add annotation
                coco_data["annotations"].append({
                    "id": annotation_id,
                    "image_id": image_id,
                    "category_id": class_id,
                    "bbox": bbox,
                    "area": area,
                    "segmentation": [segmentation],
                    "iscrowd": 0
                })
                
                annotation_id += 1
        
        # Save COCO JSON file with standard naming
        output_file = annotations_dir / f"instances_{split_name}.json"
        with open(output_file, 'w') as f:
            json.dump(coco_data, f, indent=2)
        
        print(f"\nSaved annotations to {output_file}")
        print(f"  Images: {len(coco_data['images'])}")
        print(f"  Annotations: {len(coco_data['annotations'])}")
        print(f"  Categories: {len(coco_data['categories'])}")
        print(f"Copied images to {split_images_dir}")
  
    return output_root


if __name__ == "__main__":
    # Paths
    data_yaml_path = "/kaggle/working/data.yaml"
    output_root = "/kaggle/working/coco_dataset" 
    

    dataset_root = convert_yolo_to_coco(data_yaml_path, output_root)

Dataset path: /kaggle/working/data
Number of classes: 5
Classes: ['dentigeroz kist', 'keratokist', 'radikuler kist', 'ameloblastoma', 'odontoma']

Creating COCO dataset at: /kaggle/working/coco_dataset
  images/
  annotations/

Processing train split...
Source images: /kaggle/working/data/train/images
Source labels: /kaggle/working/data/train/labels
Destination images: /kaggle/working/coco_dataset/images/train
Found 1005 images


Converting train: 100%|██████████| 1005/1005 [00:03<00:00, 257.45it/s]



Saved annotations to /kaggle/working/coco_dataset/annotations/instances_train.json
  Images: 1005
  Annotations: 1036
  Categories: 5
Copied images to /kaggle/working/coco_dataset/images/train

Processing val split...
Source images: /kaggle/working/data/valid/images
Source labels: /kaggle/working/data/valid/labels
Destination images: /kaggle/working/coco_dataset/images/val
Found 97 images


Converting val: 100%|██████████| 97/97 [00:00<00:00, 103.35it/s]



Saved annotations to /kaggle/working/coco_dataset/annotations/instances_val.json
  Images: 97
  Annotations: 104
  Categories: 5
Copied images to /kaggle/working/coco_dataset/images/val

Processing test split...
Source images: /kaggle/working/data/test/images
Source labels: /kaggle/working/data/test/labels
Destination images: /kaggle/working/coco_dataset/images/test
Found 98 images


Converting test: 100%|██████████| 98/98 [00:02<00:00, 45.25it/s] 


Saved annotations to /kaggle/working/coco_dataset/annotations/instances_test.json
  Images: 98
  Annotations: 103
  Categories: 5
Copied images to /kaggle/working/coco_dataset/images/test


In [5]:

!git clone https://github.com/RT-DETRs/RT-DETRv4.git
%cd RT-DETRv4

!pip install -q -r requirements.txt

!pip install -q pyyaml opencv-python pillow tqdm

Cloning into 'RT-DETRv4'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 178 (delta 27), reused 23 (delta 23), pack-reused 133 (from 1)
Receiving objects: 100% (178/178), 5.72 MiB | 28.71 MiB/s, done.
Resolving deltas: 100% (48/48), done.
/kaggle/working/RT-DETRv4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━

In [7]:

# Clone DINOv3
!git clone https://github.com/facebookresearch/dinov3.git

# Download DINOv3 weights
import os
os.makedirs('pretrain', exist_ok=True)

# Download ViT-B/16-LVD-1689M checkpoint
!wget -P pretrain/ https://dl.fbaipublicfiles.com/dinov3/dinov3_vitb16_pretrain_lvd1689m.pth


Cloning into 'dinov3'...
remote: Enumerating objects: 538, done.
remote: Counting objects: 100% (363/363), done.
remote: Compressing objects: 100% (264/264), done.
remote: Total 538 (delta 201), reused 99 (delta 99), pack-reused 175 (from 1)
Receiving objects: 100% (538/538), 9.88 MiB | 26.63 MiB/s, done.
Resolving deltas: 100% (223/223), done.
--2025-12-15 09:45:35--  https://dl.fbaipublicfiles.com/dinov3/dinov3_vitb16_pretrain_lvd1689m.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.51, 3.163.189.96, 3.163.189.14, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.51|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2025-12-15 09:45:35 ERROR 403: Forbidden.



In [8]:
# DINOv3 weights'i Hugging Face'den indir
!pip install -q huggingface_hub

from huggingface_hub import hf_hub_download

os.makedirs('pretrain', exist_ok=True)

# DINOv3 ViT-B/16 model
hf_hub_download(
    repo_id="facebook/dinov2-base", 
    filename="pytorch_model.bin",
    local_dir="pretrain/",
    local_dir_use_symlinks=False
)

# Dosyayı yeniden adlandır
!mv pretrain/pytorch_model.bin pretrain/dinov3_vitb16_pretrain_lvd1689m.pth

print("✓ DINOv3 weights downloaded from Hugging Face!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

✓ DINOv3 weights downloaded from Hugging Face!


In [6]:

import json

coco_dataset_path = '/kaggle/working/coco_dataset'
assert os.path.exists(coco_dataset_path), "Please run yolo_polygon_to_coco.py first!"


train_ann = f'{coco_dataset_path}/annotations/instances_train.json'
val_ann = f'{coco_dataset_path}/annotations/instances_val.json'

with open(train_ann, 'r') as f:
    train_data = json.load(f)

with open(val_ann, 'r') as f:
    val_data = json.load(f)

num_classes = len(train_data['categories'])
class_names = [cat['name'] for cat in train_data['categories']]

print(f"✓ Dataset found at: {coco_dataset_path}")
print(f"  Train images: {len(train_data['images'])}")
print(f"  Train annotations: {len(train_data['annotations'])}")
print(f"  Val images: {len(val_data['images'])}")
print(f"  Val annotations: {len(val_data['annotations'])}")
print(f"  Number of classes: {num_classes}")
print(f"  Classes: {class_names}")


✓ Dataset found at: /kaggle/working/coco_dataset
  Train images: 1005
  Train annotations: 1036
  Val images: 97
  Val annotations: 104
  Number of classes: 5
  Classes: ['dentigeroz kist', 'keratokist', 'radikuler kist', 'ameloblastoma', 'odontoma']


In [7]:

import yaml

# Create custom dataset config
custom_dataset_config = {
    'task': 'detection',
    'evaluator': {
        'type': 'CocoEvaluator',
        'iou_types': ['bbox']
    },
    'num_classes': 5,
    'remap_mscoco_category': False,  # IMPORTANT: For custom dataset
    
    'train_dataloader': {
        'type': 'DataLoader',
        'dataset': {
            'type': 'CocoDetection',
            'img_folder': f'{coco_dataset_path}/images/train',
            'ann_file': f'{coco_dataset_path}/annotations/instances_train.json',
            'return_masks': False,
            'transforms': {
                'type': 'Compose',
                'ops': None
            }
        },
        'shuffle': True,
        'num_workers': 4,
        'drop_last': True,
        'collate_fn': {
            'type': 'BatchImageCollateFunction'
        }
    },
    
    'val_dataloader': {
        'type': 'DataLoader',
        'dataset': {
            'type': 'CocoDetection',
            'img_folder': f'{coco_dataset_path}/images/val',
            'ann_file': f'{coco_dataset_path}/annotations/instances_val.json',
            'return_masks': False,
            'transforms': {
                'type': 'Compose',
                'ops': None
            }
        },
        'shuffle': False,
        'num_workers': 4,
        'drop_last': False,
        'collate_fn': {
            'type': 'BatchImageCollateFunction'
        }
    }
}

# Save config
os.makedirs('configs/dataset', exist_ok=True)
config_path = 'configs/dataset/dental_detection.yml'
with open(config_path, 'w') as f:
    yaml.dump(custom_dataset_config, f, default_flow_style=False)

print(f"✓ Custom dataset config saved: {config_path}")


✓ Custom dataset config saved: configs/dataset/dental_detection.yml


In [14]:
%cd RT-DETRv4

/kaggle/working/RT-DETRv4


In [8]:

import yaml

# X model için config
base_config = 'configs/rtv4/rtv4_hgnetv2_x_coco.yml'  # s → x
custom_config = 'configs/rtv4/rtv4_hgnetv2_x_dental.yml'  # s → x

# Read base config
with open(base_config, 'r') as f:
    config = yaml.safe_load(f)

# Modify for custom dataset
config['includes'] = [
    'configs/base/dataloader.yml',
    'configs/base/optimizer.yml',
    'configs/base/rtv4_base.yml',
    'configs/dataset/dental_detection.yml',
]

# Teacher model yok ise kaldır (403 hatası varsa)
if 'teacher_model' in config:
    config['teacher_model'] = None  # veya teacher olmadan eğit

# Epochs
if 'epochs' in config:
    config['epochs'] = 100

# Save
with open(custom_config, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"✓ Config saved: {custom_config}")
print(f"  Model: RT-DETRv4-X")
print(f"  Epochs: {config.get('epochs', 'default')}")

✓ Config saved: configs/rtv4/rtv4_hgnetv2_x_dental.yml
  Model: RT-DETRv4-X
  Epochs: default


In [ ]:
pip install tensorboard==2.15.2


In [9]:
!pip uninstall -y numpy
!pip install numpy==1.26.4


Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 104.4 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.5 which is 

In [14]:
!ls /kaggle/working

best.pt  coco_dataset  data  data.yaml	RT-DETRv4


In [16]:
!rm -rf /kaggle/working/data

In [ ]:
import yaml

# RT-DETRv4-X - batch=4, imgsz=896, diğer her şey orijinal
custom_config = {
    '__include__': [
        
        '../dfine/dfine_hgnetv2_x_coco.yml',  # X model
        '../base/rtv4.yml'
    ],

    'output_dir': './outputs/rtv4_datav2_pretrain',

    # -------------------------------------------------
    # Dataset
    # -------------------------------------------------
    'num_classes': 5,
    'remap_mscoco_category': False,

    # -------------------------------------------------
    # Train dataloader
    # -------------------------------------------------
    'train_dataloader': {
        'total_batch_size': 4,
        'num_workers': 2,
        'dataset': {
            'img_folder': '/kaggle/working/coco_dataset/images/train',
            'ann_file': '/kaggle/working/coco_dataset/annotations/instances_train.json',
            'transforms': {
                'ops': [
                    {'type': 'RandomPhotometricDistort', 'p': 0.3},  # Hafif
                    {'type': 'Resize', 'size': [896, 896]}, 
                    {'type': 'SanitizeBoundingBoxes', 'min_size': 1},
                    {'type': 'ConvertPILImage', 'dtype': 'float32', 'scale': True},
                    {'type': 'ConvertBoxes', 'fmt': 'cxcywh', 'normalize': True}
                ],
                'policy': {
                    'name': 'stop_epoch',
                    'epoch': [4, 25, 38],  # Kısa mosaic schedule
                    'ops': ['Mosaic']
                },
                'mosaic_prob': 0.0  # Hafif mosaic
            },
        },
        'collate_fn': {
            'base_size': 896,
            'base_size_repeat': 0,
            'stop_epoch': 38,
            'mixup_prob': 0.0,  # Hafif mixup
            'mixup_epochs': [4, 25]
        }
    },

    # -------------------------------------------------
    # Validation dataloader
    # -------------------------------------------------
    'val_dataloader': {
        'total_batch_size': 4,
        'num_workers': 2,
        'dataset': {
            'img_folder': '/kaggle/working/coco_dataset/images/val',
            'ann_file': '/kaggle/working/coco_dataset/annotations/instances_val.json',
            'transforms': {
                'ops': [
                    {'type': 'Resize', 'size': [896, 896]},
                    {'type': 'ConvertPILImage', 'dtype': 'float32', 'scale': True}
                ]
            }
        }
    },

    # -------------------------------------------------
    # Teacher distillation kapalı (memory + stabilite)
    # -------------------------------------------------
    'HybridEncoder': {
        'distill_teacher_dim': 0
    },

    'RTv4Criterion': {
        'weight_dict': {
            'loss_distill': 0
        },
        'losses': ['mal', 'boxes', 'local']
    },

    # -------------------------------------------------
    # Optimizer (batch=4 için uygun)
    # -------------------------------------------------
    'optimizer': {
        'lr': 3e-5,
        'weight_decay': 0.0001,
    },


    # -------------------------------------------------
    # Training
    # -------------------------------------------------
    'epoches': 60,
    'eval_spatial_size': [896, 896],
    'save_checkpoint_interval': 20,

    'pretrained': 'outputs/rtv4_datav2_pretrain/best_stg1.pth',
    'resume': False,
}

# -------------------------------------------------
# Config dosyasını yaz
# -------------------------------------------------
config_path = 'configs/rtv4/rtv4_hgnetv2_x_dental_896.yml'
with open(config_path, 'w') as f:
    yaml.dump(custom_config, f, default_flow_style=False, sort_keys=False)

print("✓ RT-DETRv4-X Config hazır")
print("  Model: X (largest)")
print("  Batch: 4")
print("  Image: 896x896")
print("  Augmentation: Orijinal")
print("  Epochs: 100")

# -------------------------------------------------
# GPU temizle
# -------------------------------------------------
import gc, torch
gc.collect()
torch.cuda.empty_cache()

# -------------------------------------------------
# Eğit
# -------------------------------------------------
!torchrun --nproc_per_node=1 train.py \
    -c configs/rtv4/rtv4_hgnetv2_x_dental_896.yml \
    --use-amp \
    --seed=0 \



✓ RT-DETRv4-X Config hazır
  Model: X (largest)
  Batch: 4
  Image: 896x896
  Augmentation: Orijinal
  Epochs: 100
2025-12-15 13:54:13.395582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765806853.416740     702 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765806853.423629     702 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tensorboard/compat/__init__.py", line 42, in tf
    from tensorboard.compat import notf  # noqa: F401
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ImportError: cannot import name 'notf' from 'tensorboard.compat' (/usr/local/lib/python3.11/dist-